### Assignment : Week 3
## Efficiently finding optimal policies in MABs

In this assignment, we will work with Multi Armed Bandit environments, and try to find the best policies using different strategies to minimize the total regret.

The aim of this exercise is to code agents capable of understanding the underlying probability distributions of the environment and finding the most optimal actions as early as possible.

You can start this assignment during/after reading Grokking Ch-4.

Let's get started!

In [39]:
# importing necessary stuff
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

# if you want to use envs from Gym, import it
# import gym, gym_bandits

Let's make a simple **2-armed Bernoulli** bandit.

If you want a cleaner code, you can implement Bandits using `class` in Python.

We have included sample code for this in `bandits.py` which you can take/import.

In [40]:
# generating the underlying probability distribution

probs = np.random.random(2)

In [41]:
# our MDP is a function which takes an action and returns a reward

def mab_2_env(action):
    gen = np.random.random()

    # for bernoulli bandits, the reward is 1 if the random number is less than the probability of success, else 0
    return gen < probs[action]

Let's now make a template for testing different strategies.

Feel free to modify this code.

In [42]:
def test_strategy(env, n_actions, selector, n_episodes = 1000):
    
    # initialize Q and N to 0s
    Q = np.zeros(n_actions)
    N = np.zeros(n_actions)
    rewards = 0
    actions = []
    # loop for n_episodes
    for e in range(0,n_episodes):
        
        # selector function takes in current Q and returns an action
        # modify the selector function according to the strategy
        if selector == exponentially_decaying_epsilon_greedy:
            action = selector(Q,0.5,e)
        elif selector == epsilon_greedy:
            action = selector(Q,0.1)
        elif selector == random_selector:
            action = selector(Q)

        elif selector == softmax_strategy:
            action = selector(Q,0.9,e)
        elif selector == ucb:
            action = selector(Q,N,1,e)
        elif selector == thompson_sampling:
            action = selector(Q,N,0.5,0.5)


        # get the reward from the environment
        reward = env(action)
        rewards += reward
        actions.append(action)
        # update N and Q
        N[action] += 1
        Q[action] += (reward - Q[action])/N[action]

    # return the best action
    return np.argmax(Q),rewards,actions


Implementing the simplest selector using pure-exploration strategy.

In [43]:
# selector returns a random action
random_selector = lambda Q : np.random.randint(len(Q))

Let's test this strategy.

In [ ]:
test_strategy(mab_2_env, 2, random_selector)

: 

As you can see, it returns the optimal action. Let's check if that's indeed true.

We can do that by revealing the actual `probs` distribution.

In [45]:
probs

array([0.80499935, 0.65065254])

As expected, our pure exploration strategy does indeed return the optimal action for this Bernoulli bandit. 

You can try generating new bandits with different `probs` and try out the same.

With all this in place, here's what you have to do -

Recall that, the regret $\mathcal{T}$ is given by,

$$\mathcal{T}=\sum  _{e=1} ^{E} \mathbb{E} \left[ v_* - q_* \left( A_e \right) \right]$$

We can only calculate it when we have the $v_*$ and $q_*$ functions known beforehand. Since we are making the MDPs from scratch, that's not an issue for us right now.

But remember, in real-life problems, these functions are not known. Hence we must be aware of multiple policy finding strategies and try the one which gives best results fastest.

### Todo 0

Implement the calculation of the total regret $\mathcal{T}$ for your strategy.

To do this, you will need to store the rewards obtained each episode. Modify the `strategy` function accordingly.

In [46]:
# function to calculate total regret

def regret(rewards, probs):

    # implement the regret function here
    #print(rewards)
    size = len(rewards)
    u = 0
    for i in range(0,len(probs)):
        if probs[i] > probs[u]:
            u = i
    t1 = probs[u]*size
    sum = 0
    for i in range(0,size):
        sum += probs[rewards[i]]
    #print('t1 is ',t1)
    #print('sum is ',sum)
    return t1-sum

    raise NotImplementedError()

### Todo 1

Now, let's implement some other selection strategies and compare their regret with the simple exploration strategy.

Note that some of these strategies involve hyperparameter(s) which need to be manually adjusted. You have to play around with the values and see which one gives you best results.

This is known as "hyperparameter tuning" and is quite commonly done while working with complex models (including neural networks). Personally, you should try out some natural values (including the ones given in the book) along with some extreme values where it is easy to manually verify the correctness of your strategy.

In [47]:
# epsilon-greedy strategy
# Already implemented for you coz I am nice

def epsilon_greedy(Q, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(len(Q))
    else:
        return np.argmax(Q)

In [48]:
# exponentially decaying epsilon greedy strategy

def exponentially_decaying_epsilon_greedy(Q, epsilon, episode):
    
    # implement the exponentially decaying epsilon greedy strategy here

    nums = epsilon**(episode - 1)
    if np.random.random() < nums:
        return np.random.randint(len(Q))
    else:
        return np.argmax(Q)

    raise NotImplementedError()


In [49]:
# softmax action selection strategy

def softmax_strategy(Q, temperature,episode):
    
    # implement the softmax strategy here
    if episode == 1:
        return np.random.randint(len(Q))
    nums = temperature**(episode - 1)
    if nums < 0.001:
        nums = 0.001
    scaledQ = Q/nums
    normQ = scaledQ - np.max(scaledQ)
    expQ = np.exp(normQ)
    prob = expQ/np.sum(expQ)
    ac = np.random.choice(np.arange(len(prob)),size=1,p=prob)[0]
    return ac
    raise NotImplementedError()

In [50]:
# upper confidence bound strategy

def ucb(Q, N, c, episode):
    
    # implement the ucb strategy here
    for i in range(0,len(N)):
        if N[i] == 0:
            return i
    arr = 1/N
    arr = arr*np.log(episode)
    arr = np.sqrt(arr)
    arr = arr*c
    arr = arr + Q
    return np.argmax(arr)
    

In [51]:
# thompson sampling strategy

def thompson_sampling(Q, N, alpha, beta):
    
    # implement the thompson sampling strategy here
    for i in range(0,len(N)):
        if N[i] == 0:
            return i
    samples = np.random.normal(loc=Q, scale=alpha/(np.sqrt(N) + beta))
    ac = np.argmax(samples)
    return ac
    raise NotImplementedError()


### Todo 2

Run each strategy for 2-armed bandit environment and compare the total regrets.

You can also try plotting the regret vs episode graph and check if it matches the expected result from Grokking

In [52]:
# import plotting libraries

import matplotlib.pyplot as plt

### Todo 3

The 2-armed bandits might be too simple for us to actually see substantial difference in the regret of these strategies. 

Let's now create a more complicated bandit environment and replicate our results on it.

We will now implement a 10-armed Gaussian bandit. 

As required, it will have possible actions and each action will generate a reward sampled from a Gaussian distribution.

Hence, each "arm" will have a randomly generated $\mu$ and $\sigma$, and the rewards will be generated with probabilities following the $\mathcal{N}(\mu, \sigma^2)$ distribution. 

In [53]:
# 10 arm gaussian bandit
# generate the means for each arm
means = np.random.rand(10)

# generate the variance for each arm
variances = np.random.rand(10)

In [54]:
# our MDP is a again function which takes an action and returns a reward

def mab_10_env(action):

    # for gaussian bandits, the reward is generated from a normal distribution
    return np.random.normal(means[action],np.sqrt(variances[action]))
    raise NotImplementedError()

### Todo 4

Test the different strategies on the 10-armed gaussian bandit and verify your results.